## Using logprobs to ensure that an acrostic poem is generated

In [5]:
%pip install --upgrade --quiet transformers torch fbgemm-gpu accelerate

Note: you may need to restart the kernel to use updated packages.


In [1]:
# CHANGE this to the Llama model for which you have applied for access via Hugging Face
# See: https://www.llama.com/docs/getting-the-models/hugging-face/
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

In [2]:
import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["HF_TOKEN"][:2] == "hf",\
       "Please sign up for access to the specific Llama model via HuggingFace and provide access token in keys.env file"

## Zero-shot generation


In [4]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation", 
    model=MODEL_ID,
    use_fast=True,
    kwargs={
        "return_full_text": False,
    },
    model_kwargs={}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [5]:
def get_input_message(animal: str, poem_so_far=None):
    system_prompt="""
    You are a poet writing acrostic poems about animals for a childrens' book.
    The first letters of the acrostic poem need to spell out an adjective that is suitable for the animal.
    Each line needs to be a phrase that describes the animal.
    For example, the first letters of an acrostic poem about rabbits might spell out "quick" or "cute"
    and the entire poem needs to be a single phrase.

    For example, an acrostic poem about rabbits might be:

    Quietly, the rabbit bides its time
    Under the garden deck
    In wait for
    Carrot greens,
    Kale, and parsley.
    """
 
    if poem_so_far:
        user_prompt=f"""
        Complete the following acrostic poem about {animal}:
        
        {poem_so_far}
        """
    else:
        user_prompt=f"""
        Write an acrostic poem about {animal}.  Respond with only the poem. Do not include any preamble or introduction.
        """
        
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [ ]:
results = pipe(get_input_message("tiger"), max_new_tokens=256)

In [11]:
print(results)

[{'generated_text': [{'role': 'system', 'content': '\n    You are a poet writing acrostic poems about animals for a childrens\' book.\n    The first letters of the acrostic poem need to spell out an adjective that is suitable for the animal.\n    Each line needs to be a phrase that describes the animal.\n    For example, the first letters of an acrostic poem about rabbits might spell out "quick" or "cute"\n    and the entire poem needs to be a single phrase.\n\n    For example, an acrostic poem about rabbits might be:\n\n    Quietly, the rabbit bides its time\n    Under the garden deck\n    In wait for\n    Carrot greens,\n    Kale, and parsley.\n    '}, {'role': 'user', 'content': '\n        Write an acrostic poem about tiger.  Respond with only the poem. Do not include any preamble or introduction.\n        '}, {'role': 'assistant', 'content': 'Terrific eyes scan the night\nRuling forests with gentle might\nIncredible strength in every stride\nNimble paws, a silent guide\nElegant bea

In [12]:
print(results[0]['generated_text'][-1]['content'])

Terrific eyes scan the night
Ruling forests with gentle might
Incredible strength in every stride
Nimble paws, a silent guide
Elegant beauty, a wondrous sight


As you can see, the model has a problem generating TIGER as the starting letters.

## Using the logprobs to control the generation

In [ ]:
results = pipe(get_input_message("tiger"), max_new_tokens=256, output_scores=True)
print(results[0]['generated_text'][-1]['content'])